In [1]:
# https://rasterio.readthedocs.io/en/latest/

import rasterio
import rasterio.features
import rasterio.warp

with rasterio.open('../preprocessed_data/dimensionality_reduction/cross_lochs.tif') as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:32630', geom, precision=6)
        
# if change to EPSG:4326, then will get 'normal' lat long coordinates

        # Print GeoJSON shapes to stdout.
        print(geom)

{'type': 'Polygon', 'coordinates': [[[444730.79093, 6472606.53824], [444730.79093, 6471576.53824], [445735.79093, 6471576.53824], [445735.79093, 6472606.53824], [444730.79093, 6472606.53824]]]}


In [3]:
import pandas as pd
import geopandas
import rasterio as rio
cross_lochs_test = rio.open('../preprocessed_data/dimensionality_reduction/cross_lochs_2.tif')
# number of bands
print(cross_lochs_test.count)

# https://rasterio.readthedocs.io/en/latest/quickstart.html
print(cross_lochs_test.width)
print(cross_lochs_test.height)
print(cross_lochs_test.bounds)

print(cross_lochs_test.transform)
lat_long_NW = cross_lochs_test.transform * (0, 0)
print(lat_long_NW)
lat_long_SE = cross_lochs_test.transform * (cross_lochs_test.width, cross_lochs_test.height)
print(lat_long_SE)
print(cross_lochs_test.crs)

# read bands
array = cross_lochs_test.read()

# convert to a DataFrame
# import pandas as pd

cross_lochs_df = pd.DataFrame()

array_num = 0
band_num = 1
min_lat = lat_long_NW[0]
min_long = lat_long_SE[1]
max_lat = lat_long_SE[0]
max_long = lat_long_NW[1]

# want to add in spatial indexing - see website

for num in range(358):
    cross_lochs_df['band' +str(band_num)]=array[array_num].ravel()
    array_num += 1
    band_num += 1

    

# cross_lochs_df_located = geopandas.GeoDataFrame(cross_lochs_df, geometry=geopandas.points_from_xy(cross_lochs_df.Longitude, cross_lochs_df.Latitude))
# won't work as lat and long aren't returned in the table

# may be able to do using rasterio documentation - looks likely, it's just working out how to add the specific indicies.
# there is also info on how to save a raster


cross_lochs_df.head()

358
203
207
BoundingBox(left=444721.807637, bottom=6471573.15507, right=445736.807637, top=6472608.15507)
| 5.00, 0.00, 444721.81|
| 0.00,-5.00, 6472608.16|
| 0.00, 0.00, 1.00|
(444721.807637, 6472608.15507)
(445736.807637, 6471573.15507)
EPSG:32630


/tmp/ipykernel_24431/3868988842.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cross_lochs_df['band' +str(band_num)]=array[array_num].ravel()


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,0.015441,...,0.077946,0.077413,0.077153,0.078934,0.074587,0.072792,0.075150,0.074088,0.074698,0.072249
1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,0.015468,...,0.075149,0.073557,0.071610,0.071727,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113
2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,0.017382,...,0.088601,0.087386,0.087973,0.089698,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678
3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,0.018532,...,0.105301,0.103799,0.104660,0.107808,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883
4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,0.017144,...,0.095351,0.096855,0.094500,0.093532,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215


In [4]:
cross_lochs_df["POINTID"] = ""

# a final column of POINTID has been added (checked)

index_num = 0
point_num = 1

for index, row in cross_lochs_df.iterrows():
    cross_lochs_df.iloc[index_num, 358] = point_num
    point_num += 1
    index_num += 1
    
cross_lochs_df.head()

/tmp/ipykernel_24431/3967678976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cross_lochs_df["POINTID"] = ""


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band350,band351,band352,band353,band354,band355,band356,band357,band358,POINTID
0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,0.015441,...,0.077413,0.077153,0.078934,0.074587,0.072792,0.075150,0.074088,0.074698,0.072249,1
1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,0.015468,...,0.073557,0.071610,0.071727,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2
2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,0.017382,...,0.087386,0.087973,0.089698,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3
3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,0.018532,...,0.103799,0.104660,0.107808,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4
4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,0.017144,...,0.096855,0.094500,0.093532,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5


In [5]:
geometry_file = pd.read_csv('../preprocessed_data/dimensionality_reduction/shape/cross_lochs_geometry.csv')

geometry_file

,wkt_geom,POINTID,GRID_CODE
0,POINT (444724.30763699999079108 6472605.655070...,1,0.001099
1,POINT (444729.30763699999079108 6472605.655070...,2,0.003365
2,POINT (444734.30763699999079108 6472605.655070...,3,0.002603
3,POINT (444739.30763699999079108 6472605.655070...,4,0.002381
4,POINT (444744.30763699999079108 6472605.655070...,5,0.002342
...,...,...,...
42016,POINT (445714.30763699999079108 6471575.655070...,42017,0.001845
42017,POINT (445719.30763699999079108 6471575.655070...,42018,0.002455
42018,POINT (445724.30763699999079108 6471575.655070...,42019,0.002387
42019,POINT (445729.30763699999079108 6471575.655070...,42020,0.002777


In [6]:
cross_lochs_gdf = pd.merge(cross_lochs_df, geometry_file, on="POINTID")

cross_lochs_gdf = cross_lochs_gdf.rename(columns={"wkt_geom": "geometry"})

cross_lochs_gdf.head()

# Managed to add geometry to the file - can it now be used? 
# If can, can then try adding another column of PFTs to then be able to train data

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band352,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE
0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,0.015441,...,0.078934,0.074587,0.072792,0.075150,0.074088,0.074698,0.072249,1,POINT (444724.30763699999079108 6472605.655070...,0.001099
1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,0.015468,...,0.071727,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.30763699999079108 6472605.655070...,0.003365
2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,0.017382,...,0.089698,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3,POINT (444734.30763699999079108 6472605.655070...,0.002603
3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,0.018532,...,0.107808,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4,POINT (444739.30763699999079108 6472605.655070...,0.002381
4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,0.017144,...,0.093532,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5,POINT (444744.30763699999079108 6472605.655070...,0.002342


In [7]:
from shapely.geometry import Point
#gdf = geopandas.GeoDataFrame(cross_lochs_gdf)

# gdf.set_crs(epsg=32630)

cross_lochs_gdf['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs_gdf['geometry'])

gdf = geopandas.GeoDataFrame(cross_lochs_gdf, geometry='geometry')

# one issue was Point not POINT



# gdf.crs is None



# option 1: (Attribute error - 'Series' object has no attribute 'centroid')
# https://stackoverflow.com/questions/49635436/shapely-point-geometry-in-geopandas-df-to-lat-lon-columns
# gdf['Center_point'] = gdf['geometry'].centroid
# #Extract lat and lon from the centerpoint
# gdf["long"] = gdf.Center_point.map(lambda p: p.x)
# gdf["lat"] = gdf.Center_point.map(lambda p: p.y)

# option 2: (AttributeError: 'Series' object has no attribute 'set_crs' or 'to_crs')
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html - not original website used
# gdf.set_crs(epsg=32630)
# print(gdf.crs)
# gdf = gdf.to_crs(epsg=4326)

# gdf = geopandas.GeoDataFrame(
#     cross_lochs_gdf, geometry="wkt_geom")

# gdf.explore("band353")

In [8]:
gdf.describe()

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band350,band351,band352,band353,band354,band355,band356,band357,band358,GRID_CODE
count,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,...,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000,42021.000000
mean,0.001254,0.002138,0.003960,0.005098,0.006120,0.008425,0.010456,0.012139,0.014304,0.016267,...,0.082205,0.081238,0.082423,0.080989,0.081207,0.081550,0.082234,0.080923,0.080996,0.001254
std,0.001117,0.001244,0.001505,0.001783,0.002021,0.002487,0.002752,0.003051,0.003381,0.003709,...,0.031395,0.031072,0.031140,0.030838,0.030895,0.030779,0.031334,0.031033,0.031286,0.001117
min,-0.006075,-0.007359,-0.005020,-0.004328,-0.003847,-0.002009,-0.000129,0.001061,0.002355,0.003381,...,0.001169,0.000348,0.000071,0.001599,0.000002,0.000455,0.001369,0.000981,0.000665,-0.006075
25%,0.000715,0.001597,0.003320,0.004423,0.005416,0.007676,0.009666,0.011296,0.013391,0.015340,...,0.075577,0.074487,0.075289,0.073723,0.073641,0.073624,0.074156,0.072663,0.072637,0.000715
50%,0.001101,0.002115,0.004162,0.005446,0.006603,0.009087,0.011217,0.012998,0.015277,0.017352,...,0.092944,0.091896,0.093096,0.091571,0.091806,0.092107,0.092989,0.091604,0.091780,0.001101
75%,0.001599,0.002521,0.004686,0.006058,0.007290,0.009899,0.012100,0.013984,0.016342,0.018525,...,0.101737,0.100699,0.101977,0.100478,0.100755,0.101201,0.102164,0.100762,0.101010,0.001599
max,0.009608,0.013708,0.012831,0.017133,0.016094,0.019169,0.022865,0.024239,0.028413,0.029250,...,0.156520,0.151882,0.154203,0.154168,0.153671,0.153331,0.157076,0.154964,0.154738,0.009608


In [ ]:
gdf.to_file("cross_lochs_gdf.shp")